Here we create a table for the "static part" of the Case National file: the numbers of Covid-19 infected people per day.

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

In [ ]:
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [object_id] = OBJECT_ID('[VWSINTER].[RIVM_COVID_19_CASE_NATIONAL]'))
CREATE TABLE [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL](
	[ID] [bigint] IDENTITY (1,1) NOT NULL,
	[DATE_FILE] [datetime] NULL,
	[DATE_STATISTICS] [datetime] NULL,
	[DATE_STATISTICS_TYPE] [varchar](100) NULL,
	[AGEGROUP] [varchar](100) NULL,
	[SEX] [varchar](100) NULL,
	[PROVINCE] [varchar](100) NULL,
	[HOSPITAL_ADMISSION] [varchar](100) NULL,
	[DECEASED] [varchar](100) NULL,
	[WEEK_OF_DEATH] [varchar](100) NULL,
	[MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
	[DATE_LAST_INSERTED] [datetime]  DEFAULT GETDATE()
) ON [PRIMARY]
GO

--Check if [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] has identity column
IF NOT EXISTS (SELECT * from syscolumns where id = Object_ID('[VWSINTER].[RIVM_COVID_19_CASE_NATIONAL]') and colstat & 1 = 1)
BEGIN
 
		--else drop seq and table and create the table in a correct way
		DROP TABLE IF EXISTS [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL]
	
		SET QUOTED_IDENTIFIER ON
		SET ARITHABORT ON
		SET NUMERIC_ROUNDABORT OFF
		SET CONCAT_NULL_YIELDS_NULL ON
		SET ANSI_NULLS ON
		SET ANSI_PADDING ON
		SET ANSI_WARNINGS ON
		
		
		CREATE TABLE [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL](
			[ID] [bigint] IDENTITY (1,1) NOT NULL,
			[DATE_FILE] [datetime] NULL,
			[DATE_STATISTICS] [datetime] NULL,
			[DATE_STATISTICS_TYPE] [varchar](100) NULL,
			[AGEGROUP] [varchar](100) NULL,
			[SEX] [varchar](100) NULL,
			[PROVINCE] [varchar](100) NULL,
			[HOSPITAL_ADMISSION] [varchar](100) NULL,
			[DECEASED] [varchar](100) NULL,
			[WEEK_OF_DEATH] [varchar](100) NULL,
			[MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
			[DATE_LAST_INSERTED] [datetime]  DEFAULT GETDATE()
		) ON [PRIMARY]

END




In [ ]:
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [object_id] = OBJECT_ID('[VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL]'))
CREATE TABLE [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL](
	[ID] [int] IDENTITY (1,1) NOT NULL,
	[DATE_FILE] [varchar](100) NULL,
	[DATE_STATISTICS] [varchar](100) NULL,
	[DATE_STATISTICS_TYPE] [varchar](100) NULL,
	[AGEGROUP] [varchar](100) NULL,
	[SEX] [varchar](100) NULL,
	[PROVINCE] [varchar](100) NULL,
	[HOSPITAL_ADMISSION] [varchar](100) NULL,
	[DECEASED] [varchar](100) NULL,
	[WEEK_OF_DEATH] [varchar](100) NULL,
	[MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
	[DATE_LAST_INSERTED] [datetime] DEFAULT GETDATE(),
	[VERSION] [varchar](50) NULL
) ON [PRIMARY]
GO

We use the `@cut_off_date` variable to determine wich data we preserve in the "static table". The data from the last inserted file will be used and from this data we use the data with a `[DATE_STATISTICS]` that lies before the `@cut_off_date`.

In [ ]:
DECLARE @cut_off_date DATE = CAST('2021-10-03' AS DATE)

-- TRUNCATE TABLE [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL]

-- SET IDENTITY_INSERT [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] ON

INSERT INTO [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] (
	[DATE_FILE],
	[DATE_STATISTICS],
	[DATE_STATISTICS_TYPE],
	[AGEGROUP],
	[SEX],
	[PROVINCE],
	[HOSPITAL_ADMISSION],
	[DECEASED],
	[WEEK_OF_DEATH],
	[MUNICIPAL_HEALTH_SERVICE]
)
SELECT
	[DATE_FILE],
	[DATE_STATISTICS],
	[DATE_STATISTICS_TYPE],
	[AGEGROUP],
	[SEX],
	[PROVINCE],
	[HOSPITAL_ADMISSION],
	[DECEASED],
	[WEEK_OF_DEATH],
	[MUNICIPAL_HEALTH_SERVICE]
FROM 
    [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK)
WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK))
    AND CAST([dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]) AS DATE) <= @cut_off_date

-- SET IDENTITY_INSERT [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] OFF


Now we Alter the procedure that copies data from the VWSTAGE schema to the VWSINTER schema, to combine data from both the newly created (and populated) table in VWSSTATIC and the VWSSTAGE-table.

In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_COVID_19_NATIONAL_INTER]
AS
BEGIN
    -- WITH CTE As (
    --     SELECT
    --         [DATE_FILE],
    --         [DATE_STATISTICS],
    --         [DATE_STATISTICS_TYPE],
    --         [AGEGROUP],
    --         [SEX],
    --         [PROVINCE],
    --         [HOSPITAL_ADMISSION],
    --         [DECEASED],
    --         [WEEK_OF_DEATH],
    --         [MUNICIPAL_HEALTH_SERVICE]
    --     FROM
    --         [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL]
    --     WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL])
    -- )
    INSERT INTO [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] (
        [DATE_FILE],
        [DATE_STATISTICS],
        [DATE_STATISTICS_TYPE],
        [AGEGROUP],
        [SEX],
        [PROVINCE],
        [HOSPITAL_ADMISSION],
        [DECEASED],
        [WEEK_OF_DEATH],
        [MUNICIPAL_HEALTH_SERVICE]
    )
    SELECT
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_FILE]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]),
        TRIM([DATE_STATISTICS_TYPE]),
        TRIM([AGEGROUP]),
        TRIM([SEX]),
        TRIM([PROVINCE]),
        TRIM([HOSPITAL_ADMISSION]),
        TRIM([DECEASED]),
        TRIM([WEEK_OF_DEATH]),
        TRIM([MUNICIPAL_HEALTH_SERVICE])
    FROM
        [VWSSTAGE].[RIVM_COVID_19_CASE_NATIONAL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_COVID_19_CASE_NATIONAL])
    -- UNION ALL
    -- SELECT
    --     [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_FILE]),
    --     [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]),
    --     TRIM([DATE_STATISTICS_TYPE]),
    --     TRIM([AGEGROUP]),
    --     TRIM([SEX]),
    --     TRIM([PROVINCE]),
    --     TRIM([HOSPITAL_ADMISSION]),
    --     TRIM([DECEASED]),
    --     TRIM([WEEK_OF_DEATH]),
    --     TRIM([MUNICIPAL_HEALTH_SERVICE])
    -- FROM CTE
END;
GO

In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_COVID_19_NATIONAL_STATIC_TO_INTER] (
    @date_of_insertion DATETIME = GETDATE
)
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] (
        [DATE_FILE],
        [DATE_STATISTICS],
        [DATE_STATISTICS_TYPE],
        [AGEGROUP],
        [SEX],
        [PROVINCE],
        [HOSPITAL_ADMISSION],
        [DECEASED],
        [WEEK_OF_DEATH],
        [MUNICIPAL_HEALTH_SERVICE],
        [DATE_LAST_INSERTED]
    )
    SELECT
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_FILE]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]),
        TRIM([DATE_STATISTICS_TYPE]),
        TRIM([AGEGROUP]),
        TRIM([SEX]),
        TRIM([PROVINCE]),
        TRIM([HOSPITAL_ADMISSION]),
        TRIM([DECEASED]),
        TRIM([WEEK_OF_DEATH]),
        TRIM([MUNICIPAL_HEALTH_SERVICE]),
        @date_of_insertion
    FROM
        [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK))
END;
GO